In [1]:
!pip uninstall gensim
!pip install gensim==3.8.3
!pip install nltk

  Using cached scipy-1.8.0-cp39-cp39-win_amd64.whl (36.9 MB)
Using legacy 'setup.py install' for gensim, since package 'wheel' is not installed.
    Running setup.py install for gensim: started
    Running setup.py install for gensim: finished with status 'done'


You should consider upgrading via the 'c:\users\samil\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\samil\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA


In [3]:
!pip3 install pickle5
import pickle5 as pickle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = '/content/drive/MyDrive/CS48000-DesignPatterns'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
infile = open(DATA_PATH+'/bugfixes_valid.pickle','rb')
df = pickle.load(infile)

In [7]:
import smart_open

def read_corpus(fname, tokens_only=False):
        for i, line in enumerate(fname):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [20]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df['full_traceback'], test_size=0.2, random_state=0)

In [21]:
train_corpus = list(read_corpus(train))

In [22]:
train_corpus[:2]

[TaggedDocument(words=['the', 'full', 'traceback', 'is', 'traceback', 'most', 'recent', 'call', 'last', 'file', 'tmp', 'ansible_jmc', 'py', 'line', 'in', 'module', 'main', 'file', 'tmp', 'ansible_jmc', 'py', 'line', 'in', 'main', 'check_mode', 'module', 'check_mode', 'file', 'tmp', 'ansible_jmc', 'py', 'line', 'in', 'subnet', 'create_subnet', 'conn', 'module', 'vpc_id', 'cidr', 'az', 'check_mode', 'file', 'tmp', 'ansible_jmc', 'py', 'line', 'in', 'create_subnet', 'new_subnet', 'get_subnet_info', 'conn', 'create_subnet', 'vpcid', 'vpc_id', 'cidrblock', 'cidr', 'az', 'file', 'usr', 'local', 'lib', 'python', 'dist', 'packages', 'botocore', 'client', 'py', 'line', 'in', 'return', 'self', 'operation_name', 'kwargs', 'file', 'usr', 'local', 'lib', 'python', 'dist', 'packages', 'botocore', 'client', 'py', 'line', 'in', 'api_params', 'operation_model', 'context', 'request_context', 'file', 'usr', 'local', 'lib', 'python', 'dist', 'packages', 'botocore', 'client', 'py', 'line', 'in', 'api_param

In [38]:
test_corpus = list(read_corpus(test))

In [23]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [24]:
model.build_vocab(train_corpus)

In [26]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [27]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [28]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 2944, 1: 1207, 2: 644, 3: 369, 4: 259, 5: 211, 6: 185, 7: 146, 8: 107, 9: 90, 10: 79, 11: 71, 12: 63, 13: 57, 16: 44, 15: 39, 22: 34, 17: 34, 21: 33, 20: 32, 14: 32, 19: 32, 18: 21, 26: 17, 27: 17, 31: 16, 32: 16, 34: 16, 60: 14, 28: 14, 38: 14, 29: 14, 23: 13, 25: 13, 47: 13, 30: 13, 33: 12, 48: 12, 50: 11, 53: 11, 40: 11, 46: 11, 44: 11, 71: 11, 54: 10, 55: 10, 24: 10, 82: 10, 58: 9, 37: 9, 62: 9, 57: 9, 35: 9, 76: 9, 41: 9, 36: 9, 43: 9, 120: 8, 94: 8, 49: 7, 67: 7, 42: 7, 134: 7, 80: 7, 84: 7, 66: 7, 45: 7, 64: 7, 56: 7, 70: 7, 39: 6, 91: 6, 73: 6, 86: 6, 59: 6, 83: 6, 92: 6, 96: 6, 90: 5, 85: 5, 77: 5, 101: 5, 63: 5, 52: 5, 107: 5, 106: 5, 88: 4, 108: 4, 105: 4, 100: 4, 87: 4, 118: 4, 102: 4, 74: 4, 128: 4, 110: 4, 79: 4, 75: 4, 121: 4, 115: 4, 113: 4, 95: 4, 97: 4, 116: 4, 61: 4, 125: 4, 81: 4, 78: 4, 72: 3, 69: 3, 51: 3, 103: 3, 109: 3, 240: 3, 165: 3, 138: 3, 167: 3, 119: 3, 65: 3, 164: 3, 122: 3, 183: 3, 141: 3, 114: 2, 161: 2, 185: 2, 187: 2, 89: 2, 205: 2, 200: 2

In [29]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('THIRD-MOST', 2),  ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (7564): «error red exception in command cog update traceback most recent call last file coding redenv lib site packages discord ext commands core py line in wrapped ret await coro args kwargs file coding redenv lib site packages redbot cogs downloader downloader py line in await self ctx cogs cogs file coding redenv lib site packages redbot cogs downloader downloader py line in message await self file coding redenv lib site packages redbot cogs downloader downloader py line in failed_reqs await self cogs_to_update file coding redenv lib site packages redbot cogs downloader downloader py line in if not await repo req self lib_path file coding redenv lib site packages redbot cogs downloader repo_manager py line in await self file coding redenv lib site packages redbot cogs downloader repo_manager py line in stderr stderr decode strip utf codec can decode byte xc in position invalid continuation byte»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST

In [47]:

import random
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(list(test_corpus[doc_id][0]))
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(list(test_corpus[doc_id][0]))))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (1278): «traceback most recent call last file stdin line in module file pydicom pydicom dataset py line in update for key value in list dictionary items file pydicom pydicom dataset py line in return super dataset self name attributeerror dataset object has no attribute items»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (2473, 0.950778603553772): «error tribler_window root traceback most recent call last file users goqs documents tribler triblergui widgets py line in if len results subscribed typeerror nonetype object has no attribute»

MEDIAN (1155, 0.5737566947937012): «freqtrade persistence info updating trade id freqtrade error fatal exception traceback most recent call last file freqtrade freqtrade main py line in main state freqtrade worker old_state state file freqtrade freqtrade freqtradebot py line in worker min_secs min_secs file freqtrade freqtrade freqtradebot py line in result func args kwargs file freqtrade freqtrade freqt

In [ ]:
# http://csmoon-ml.com/index.php/2019/02/15/tutorial-doc2vec-and-t-sne/
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html